In [1]:
import itertools
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy.stats import kurtosis
from scipy.stats import skew

In [2]:
def get_features(y, sr, n_fft=1024, hop_length=512):
    features = {}

    features['spectral_centroid'] = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_rolloff'] = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zero_crossing_rate'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rms'] = librosa.feature.rms(y=y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['onset_strength'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['spectral_contrast'] = librosa.feature.spectral_contrast(y=y, sr=sr).ravel()
    features['spectral_bandwidth'] = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_flatness'] = librosa.feature.spectral_flatness(y=y, n_fft=n_fft, hop_length=hop_length).ravel()

    mfcc = librosa.feature.mfcc(y=y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)

    for index, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(index)] = v_mfcc.ravel()

    def get_moments(descriptors):
        result = {}

        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)

        return result

    additional_features = get_moments(features)
    additional_features['tempo'] = librosa.feature.tempo(y=y, sr=sr)[0]

    return additional_features

In [3]:
def process_genres_directory_files(genres_directory, genres):
    all_features = []

    for genre in genres:
        genre_directory = f'{genres_directory}/{genre}/wav_files'

        for file in os.listdir(genre_directory):
            file_path = f"{genre_directory}/{file}"

            y, sr = librosa.load(file_path)

            features = get_features(y, sr)

            features['label'] = genre

            all_features.append(features)

    return all_features

In [4]:
genres_directory = './previews'

In [5]:
genres = os.listdir(genres_directory)

In [6]:
genres

['forró', 'funk', 'pop', 'rap', 'rock', 'samba', 'sertanejo']

In [7]:
%%time

all_features = process_genres_directory_files(genres_directory, genres)

CPU times: total: 12min 35s
Wall time: 13min 34s


In [8]:
df_features = pd.DataFrame(all_features)

In [9]:
df_features.shape

(835, 128)

In [10]:
df_features.head()

,spectral_centroid_max,spectral_centroid_min,spectral_centroid_mean,spectral_centroid_std,spectral_centroid_kurtosis,spectral_centroid_skew,spectral_rolloff_max,spectral_rolloff_min,spectral_rolloff_mean,spectral_rolloff_std,...,mfcc_11_kurtosis,mfcc_11_skew,mfcc_12_max,mfcc_12_min,mfcc_12_mean,mfcc_12_std,mfcc_12_kurtosis,mfcc_12_skew,tempo,label
0,4737.508153,635.664676,2098.726133,556.496663,2.887029,1.283600,9130.078125,925.927734,4514.561770,1372.578815,...,-0.086715,-0.088010,32.658615,-31.948511,-5.576648,9.920987,-0.200280,0.234563,143.554688,forró
1,5339.076724,1019.369702,2294.895607,604.767987,1.803070,0.943630,9130.078125,1722.656250,4914.999411,1341.457851,...,0.120776,-0.207388,21.607838,-36.618702,-4.257775,9.034795,-0.080838,0.118976,143.554688,forró
2,6613.348219,1526.374864,2857.957595,685.797071,3.499504,1.245568,9474.609375,2648.583984,6027.380220,1304.601244,...,-0.253195,0.186958,32.782486,-35.293335,-7.233448,11.249509,-0.106279,0.236294,151.999081,forró
3,6935.364306,945.116845,2439.278264,730.595169,4.671441,1.223784,9840.673828,1184.326172,5551.766429,1564.960933,...,0.052664,-0.347261,28.608442,-36.924839,-3.531663,9.956684,0.135261,-0.020543,107.666016,forró
4,7086.838756,928.772862,2541.888664,744.284935,2.716824,1.082830,9969.873047,1162.792969,6118.729668,1422.390345,...,-0.254407,-0.014688,29.134048,-33.681553,-3.894992,9.132825,0.035545,0.014757,135.999178,forró


## Sobre as características do sinal de áudio

### Centroide espectral

A centroide espectral é uma medida que indica o "centro de massa" do espectro de frequência de um sinal de áudio. Em outras palavras, ela representa a média ponderada das frequências presentes no sinal, onde as frequências mais altas recebem um peso maior.

### Rolloff

o rolloff espectral é definido como a frequência abaixo da qual uma certa porcentagem (geralmente 85%) da energia total do sinal de áudio está contida nas componentes de frequência mais baixas. Essa medida é útil na análise de áudio, pois pode ser usada para inferir informações sobre a "brilho" ou "timbre" do som.

### Taxa de cruzamento por zero

A taxa de cruzamento por zero é uma medida que indica a quantidade de vezes que o sinal atravessa o eixo horizontal (valor zero) em relação ao tempo. Ela pode ser muito útil ao indicar informações sobre "aspereza" ou "textura".

### RMS

A raiz quadrada média (RMS) é uma medida que indica a energia média do sinal de áudio.

### Força do ataque (onset strength)

A força do ataque é uma medida que indica a intensidade ou saliência de mudanças de atividade rítmica no sinal de áudio.

### Contraste espectral de um sinal

O contraste espectral é uma medida que indica a relação entre a energia espectral em bandas de frequência adjacentes.

### Largura de banda espectral

A largura de banda espectral é uma medida que indica a extensão das frequências presentes no sinal de áudio.

In [11]:
df_features.to_csv('./all_features_BR.csv', index=False)